In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

user_sessions = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "user_sessions") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

post_views = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "post_views") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [8]:
distinct_session = user_sessions\
    .join(post_views,user_sessions.session_id == post_views.session_id,how="inner")\
    .withColumn("diff",col("session_endtime").cast("long")-col("session_starttime").cast("long"))\
    .select(user_sessions.session_id,col("diff"))\
    .distinct()

distinct_session\
    .join(post_views,distinct_session.session_id == post_views.session_id,how="inner")\
    .groupBy(col("post_id"))\
    .agg(sum((col("perc_viewed")/100)*col("diff")).alias("total_view_time"))\
    .filter(col("total_view_time")>5)\
    .show()

+-------+---------------+
|post_id|total_view_time|
+-------+---------------+
|      4|            5.1|
|      2|           24.0|
+-------+---------------+

